In [ ]:
from pathlib import Path
from kedro.framework.context import load_context

current_dir = Path.cwd()  # this points to 'notebooks/' folder
proj_path = current_dir.parent  # point back to the root of the project
context = load_context(proj_path)

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%reload_kedro


In [ ]:
    def _calculate_expiration_hours(expiration ) -> int:
        regex = r"(\d*)\s*([H|M|D]?)"
        try:
            num, letter = re.findall(regex,'8 H')[0]
        except:
            DataSetError('Invalid expiration')
        conversion =  {'H': 1, 'D': 24, 'M': 1/60}
        return int(num * conversion.get(letter, 24))
    
    def _calculate_file_age(filepath: Path) -> int:
        '''
        Returns the age of a file in days
        '''
        today = datetime.datetime.today()
        modified_date = datetime.datetime.fromtimestamp(os.path.getmtime(filepath))
        return (today - modified_date).days/24

    _calculate_expiration_hours('8H')

In [ ]:
from phantasyfootballer.settings import *
from phantasyfootballer.common import combine_data_horizontal

df = context.catalog.load('scoring.half_ppr')
df_left = df[KEEPER_COLUMNS]
df_right = df_left.copy()
df_right['NEW_COL'] = 'new_data'
combine_data_horizontal(df_left,df_right)

In [ ]:
from phantasyfootballer.settings import *
from phantasyfootballer.common import Stats
data = context.catalog.load('scoring.ppr')
#data.head()


In [ ]:
data.head()

    

In [ ]:
df = context.catalog.load('ppr.percent_mean_data')
df2 = context.catalog.load('ppr.percent_typical_data')
df_qb = df.query('position=="QB"')
df_wr = df.query('position=="WR"')
df_rb = df.query('position=="RB"')
df_te = df.query('position=="TE"')


len(df_qb)
len(df_rb)
len(df_te)
len(df_wr)
df_qb.columns


In [ ]:
#Distribution of players

In [ ]:
df_filter = pd.DataFrame({'A':range(100,105)})
m = df_qb.index.isin(df_filter.index)
df_qb.loc[m, 'keeper'] = True
df_qb.head(10)



In [ ]:
df = pd.DataFrame({'A':range(5), 'B':range(5,10), 'C':range(5,10)})
df2= pd.DataFrame({'A':[1,2,3], 'B':range(5,8), 'C':range(5,8)})
df
df2
df.merge(df2, how='right',indicator=True)

In [ ]:
df_qb.query('is_top_player')